In [0]:
import pyspark
from pyspark.sql import SparkSession
spark:SparkSession = SparkSession.builder\
      .master("local[1]")\
      .appName("Spark")\
      .getOrCreate()

In [0]:
from pyspark.sql.functions import lit
colObj = lit("sparkbyexamples.com")

In [0]:
data=[("James",23),("Ann",40)]
df=spark.createDataFrame(data).toDF("name.fname","gender")
df.printSchema()
#root
# |-- name.fname: string (nullable = true)
# |-- gender: long (nullable = true)

# Using DataFrame object (df)
df.select(df.gender).show()
df.select(df["gender"]).show()
#Accessing column name with dot (with backticks)
df.select(df["`name.fname`"]).show()

#Using SQL col() function
from pyspark.sql.functions import col
df.select(col("gender")).show()
#Accessing column name with dot (with backticks)
df.select(col("`name.fname`")).show()

root
-- name.fname: string (nullable = true)
-- gender: long (nullable = true)

+------+
gender|
+------+
 23|
 40|
+------+

+------+
gender|
+------+
 23|
 40|
+------+

+----------+
name.fname|
+----------+
 James|
 Ann|
+----------+

+------+
gender|
+------+
 23|
 40|
+------+

+----------+
name.fname|
+----------+
 James|
 Ann|
+----------+

In [0]:
#Create DataFrame with struct using Row class
from pyspark.sql import Row
data=[Row(name="James",prop=Row(hair="black",eye="blue")),
      Row(name="Ann",prop=Row(hair="grey",eye="black"))]
df=spark.createDataFrame(data)
df.printSchema()

#Access struct column
df.select(df.prop.hair).show()
df.select(df["prop.hair"]).show()
df.select(col("prop.hair")).show()

#Access all columns from struct
df.select(col("prop.*")).show()

root
-- name: string (nullable = true)
-- prop: struct (nullable = true)
 |-- hair: string (nullable = true)
 |-- eye: string (nullable = true)

+---------+
prop.hair|
+---------+
 black|
 grey|
+---------+

+-----+
 hair|
+-----+
black|
 grey|
+-----+

+-----+
 hair|
+-----+
black|
 grey|
+-----+

+-----+-----+
 hair| eye|
+-----+-----+
black| blue|
 grey|black|
+-----+-----+

In [0]:
data=[(100,2,1),(200,3,4),(300,4,4)]
df=spark.createDataFrame(data).toDF("col1","col2","col3")

#Arthmetic operations
df.select(df.col1 + df.col2).show()
df.select(df.col1 - df.col2).show() 
df.select(df.col1 * df.col2).show()
df.select(df.col1 / df.col2).show()
df.select(df.col1 % df.col2).show()

df.select(df.col2 > df.col3).show()
df.select(df.col2 < df.col3).show()
df.select(df.col2 == df.col3).show()

+-------------+
(col1 + col2)|
+-------------+
 102|
 203|
 304|
+-------------+

+-------------+
(col1 - col2)|
+-------------+
 98|
 197|
 296|
+-------------+

+-------------+
(col1 * col2)|
+-------------+
 200|
 600|
 1200|
+-------------+

+-----------------+
 (col1 / col2)|
+-----------------+
 50.0|
66.66666666666667|
 75.0|
+-----------------+

+-------------+
(col1 % col2)|
+-------------+
 0|
 2|
 0|
+-------------+

+-------------+
(col2 > col3)|
+-------------+
 true|
 false|
 false|
+-------------+

+-------------+
(col2 < col3)|
+-------------+
 false|
 true|
 false|
+-------------+

+-------------+
(col2 = col3)|
+-------------+
 false|
 false|
 true|
+-------------+

In [0]:
data=[("James","Bond","100",None),
      ("Ann","Varsa","200",'F'),
      ("Tom Cruise","XXX","400",''),
      ("Tom Brand",None,"400",'M')] 
columns=["fname","lname","id","gender"]
df=spark.createDataFrame(data,columns)

In [0]:
#alias
from pyspark.sql.functions import expr
df.select(df.fname.alias("first_name"), \
          df.lname.alias("last_name")
   ).show()

#Another example
df.select(expr(" fname ||','|| lname").alias("fullName") \
   ).show()

+----------+---------+
first_name|last_name|
+----------+---------+
 James| Bond|
 Ann| Varsa|
Tom Cruise| XXX|
 Tom Brand| null|
+----------+---------+

+--------------+
 fullName|
+--------------+
 James,Bond|
 Ann,Varsa|
Tom Cruise,XXX|
 null|
+--------------+

In [0]:
#asc, desc to sort ascending and descending order repsectively.
df.sort(df.fname.asc()).show()
df.sort(df.fname.desc()).show()

+----------+-----+---+------+
 fname|lname| id|gender|
+----------+-----+---+------+
 Ann|Varsa|200| F|
 James| Bond|100| null|
 Tom Brand| null|400| M|
Tom Cruise| XXX|400| |
+----------+-----+---+------+

+----------+-----+---+------+
 fname|lname| id|gender|
+----------+-----+---+------+
Tom Cruise| XXX|400| |
 Tom Brand| null|400| M|
 James| Bond|100| null|
 Ann|Varsa|200| F|
+----------+-----+---+------+

In [0]:
#cast
df.select(df.fname,df.id.cast("int")).printSchema()

root
-- fname: string (nullable = true)
-- id: integer (nullable = true)

In [0]:
#between
df.filter(df.id.between(100,300)).show()

+-----+-----+---+------+
fname|lname| id|gender|
+-----+-----+---+------+
James| Bond|100| null|
 Ann|Varsa|200| F|
+-----+-----+---+------+

In [0]:
#contains
df.filter(df.fname.contains("Cruise")).show()

+----------+-----+---+------+
 fname|lname| id|gender|
+----------+-----+---+------+
Tom Cruise| XXX|400| |
+----------+-----+---+------+

In [0]:
#startswith, endswith()
df.filter(df.fname.startswith("T")).show()
df.filter(df.fname.endswith("Cruise")).show()

+----------+-----+---+------+
 fname|lname| id|gender|
+----------+-----+---+------+
Tom Cruise| XXX|400| |
 Tom Brand| null|400| M|
+----------+-----+---+------+

+----------+-----+---+------+
 fname|lname| id|gender|
+----------+-----+---+------+
Tom Cruise| XXX|400| |
+----------+-----+---+------+

In [0]:
#isNull & isNotNull
df.filter(df.lname.isNull()).show()
df.filter(df.lname.isNotNull()).show()

+---------+-----+---+------+
 fname|lname| id|gender|
+---------+-----+---+------+
Tom Brand| null|400| M|
+---------+-----+---+------+

+----------+-----+---+------+
 fname|lname| id|gender|
+----------+-----+---+------+
 James| Bond|100| null|
 Ann|Varsa|200| F|
Tom Cruise| XXX|400| |
+----------+-----+---+------+

In [0]:
#like , rlike
df.select(df.fname,df.lname,df.id) \
  .filter(df.fname.like("%om")) 

Out[16]: DataFrame[fname: string, lname: string, id: string]

In [0]:
df.select(df.fname.substr(1,2).alias("substr")).show()


+------+
substr|
+------+
 Ja|
 An|
 To|
 To|
+------+

In [0]:
#when & otherwise
from pyspark.sql.functions import when
df.select(df.fname,df.lname,when(df.gender=="M","Male") \
              .when(df.gender=="F","Female") \
              .when(df.gender==None ,"") \
              .otherwise(df.gender).alias("new_gender") \
    ).show()

+----------+-----+----------+
 fname|lname|new_gender|
+----------+-----+----------+
 James| Bond| null|
 Ann|Varsa| Female|
Tom Cruise| XXX| |
 Tom Brand| null| Male|
+----------+-----+----------+

In [0]:
#isin
li=["100","200"]
df.select(df.fname,df.lname,df.id) \
  .filter(df.id.isin(li)) \
  .show()

+-----+-----+---+
fname|lname| id|
+-----+-----+---+
James| Bond|100|
 Ann|Varsa|200|
+-----+-----+---+

In [0]:
#Create DataFrame with struct, array & map
from pyspark.sql.types import StructType,StructField,StringType,ArrayType,MapType
data=[(("James","Bond"),["Java","C#"],{'hair':'black','eye':'brown'}),
      (("Ann","Varsa"),[".NET","Python"],{'hair':'brown','eye':'black'}),
      (("Tom Cruise",""),["Python","Scala"],{'hair':'red','eye':'grey'}),
      (("Tom Brand",None),["Perl","Ruby"],{'hair':'black','eye':'blue'})]

schema = StructType([
        StructField('name', StructType([
            StructField('fname', StringType(), True),
            StructField('lname', StringType(), True)])),
        StructField('languages', ArrayType(StringType()),True),
        StructField('properties', MapType(StringType(),StringType()),True)
     ])
df=spark.createDataFrame(data,schema)
df.printSchema()


root
-- name: struct (nullable = true)
 |-- fname: string (nullable = true)
 |-- lname: string (nullable = true)
-- languages: array (nullable = true)
 |-- element: string (containsNull = true)
-- properties: map (nullable = true)
 |-- key: string
 |-- value: string (valueContainsNull = true)

In [0]:
#getField from MapType
df.select(df.properties.getField("hair")).show()

#getField from Struct
df.select(df.name.getField("fname")).show()

+----------------+
properties[hair]|
+----------------+
 black|
 brown|
 red|
 black|
+----------------+

+----------+
name.fname|
+----------+
 James|
 Ann|
Tom Cruise|
 Tom Brand|
+----------+

In [0]:
#getItem() used with ArrayType
df.select(df.languages.getItem(1)).show()

#getItem() used with MapType
df.select(df.properties.getItem("hair")).show()

+------------+
languages[1]|
+------------+
 C#|
 Python|
 Scala|
 Ruby|
+------------+

+----------------+
properties[hair]|
+----------------+
 black|
 brown|
 red|
 black|
+----------------+